<a href="https://colab.research.google.com/github/JimKing100/Jestimate/blob/master/linear_regression_comps/LR_Comps2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [210]:
!pip install category_encoders==2.0.0
!pip install fiona
!pip install geopy

In [0]:
# Import libraries
import pandas as pd
import numpy as np
import math

import pandas_profiling

from datetime import datetime

import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.cluster import KMeans
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report

import statsmodels.api as sm

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV

from geopy.distance import vincenty as get_geodesic_distance

from bokeh.models import ColumnDataSource, TableColumn, DataTable
from bokeh.layouts import column
from bokeh.models.widgets import TextInput
from bokeh.plotting import curdoc

In [212]:
# Load SF real estate data - 10 years (2009-2018) of single family home sales in San Francisco downloaded from the SF MLS
# Longitude and latitude were added to the csv file prior to loading using geocoding.geo.census.gov 
df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Jestimate/master/data/SF-SFR-Sales-Final2d.csv')

# Rename subdistr_desc to neighborhood
df = df.rename(columns={'subdist_no': 'nid', 'subdist_desc': 'neighborhood'})

# Create subset with outliers removed - 1.6%of the data
mask = (
  (df['baths'] < 6) &
  (df['beds'] < 7) &
  (df['beds'] > 0) &
  (df['lot_sf'] < 10000) &
  (df['rooms'] < 13) &
  (df['sale_price'] < 10000000) &
  (df['sf'] < 10000) &
  (df['sf'] > 100)
)
df = df[mask]

# Check the data
print(df.shape)
df.head(5)

(19497, 39)


,longitude,latitude,elevation,full_address,city,state,street_no,street_name,street_suffix,zip,area,district_no,district_desc,nid,neighborhood,on_market_date,cdom,orig_list_price,sale_date,sale_price,rooms,baths,beds,sf_source,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",San Francisco,CA,2645,El Camino Del Mar,NaN,94121,1050,1,SF District 1,1050,1 - Outer Richmond,3/14/13,4,1095000,3/22/13,1260000,8,3.5,4,T,Per Tax Records,2691,NaN,0,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN"
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,12/3/15,144,1250000,4/25/16,1075000,9,3.0,4,T,Per Tax Records,2437,0.0689,3000,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,5/18/17,36,1395000,8/17/17,1525000,9,5.0,5,D,Per Architect,2597,0.0689,3000,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",San Francisco,CA,590,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,6/27/10,42,725000,8/20/10,715000,5,1.0,2,T,Per Tax Records,1312,NaN,0,1939,RH1,RGLR,"PVDW,PVSW",GARG,0,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK"
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",San Francisco,CA,618,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,6/7/12,24,1595000,7/13/12,1595000,8,4.0,4,T,Per Tax Records,3307,NaN,0,1951,NaN,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",0,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK"


In [0]:
#pandas_profiling.ProfileReport(df)

In [214]:
# Train, test split on date of 01/01/2018
df['sale_date'] = pd.to_datetime(df['sale_date'], infer_datetime_format=True)
df['year_sold'] = df['sale_date'].dt.year
df['month_sold'] = df['sale_date'].dt.month
df['day_sold'] = df['sale_date'].dt.day

df['on_market_date'] = pd.to_datetime(df['on_market_date'], infer_datetime_format=True)
df['year_on_market'] = df['on_market_date'].dt.year
df['month_on_market'] = df['on_market_date'].dt.month
df['day_on_market'] = df['on_market_date'].dt.day

low_cutoff = 2008
high_cutoff = 2018
train = df[(df['year_sold'] >= low_cutoff) & (df['year_sold'] < high_cutoff)]
test  = df[df['year_sold'] >= high_cutoff]
print(train.shape)
print(test.shape)

(17618, 45)
(1879, 45)


In [215]:
# Wrangle the data for train and test
def engineer_features(X):
  
  # Fill house square foot zero values with the average house square footage by bedroom for all single family homes in SF
  averagesf_data = X.groupby('beds').sf.mean()

  # Use average sf by bedroom for each 0 value in each bedroom group 1 - 6 bedrooms
  for i in range(1, 7): 
    X.loc[(X['sf'] == 0) & (X['beds'] == i), 'sf'] = averagesf_data.loc[i]
  
  # Impute mean for null long/lat/elev based on mean of of neighborhood
  def feature_calc(feature, nid, f_dict):
    if math.isnan(feature):
      if (nid) in f_dict:
        new_feature = f_dict[nid]
        return new_feature
    else:
      return feature
    
    return feature
  
  temp = X[~X['longitude'].isna()].groupby(['nid'])['longitude'].mean()
  long_dict = dict(temp)
  X['longitude'] = X.apply(lambda x: feature_calc(x['longitude'], x['nid'], long_dict), axis=1)
  
  temp = X[~X['latitude'].isna()].groupby(['nid'])['latitude'].mean()
  lat_dict = dict(temp)
  X['latitude'] = X.apply(lambda x: feature_calc(x['latitude'], x['nid'], lat_dict), axis=1)
  
  temp = X[~X['elevation'].isna()].groupby(['nid'])['elevation'].mean()
  elev_dict = dict(temp)
  X['elevation'] = X.apply(lambda x: feature_calc(x['elevation'], x['nid'], elev_dict), axis=1)
  
  X['zip'] = X['zip'].astype(int)
  
  # Fill rooms zero values by adding beds and baths
  def room_calc(rooms_val, beds_val, baths_val):
    if rooms_val == 0:
      total = beds_val + baths_val
    else:
      total = rooms_val
      
    return total
  
  X['rooms'] = X.apply(lambda x: room_calc(x['rooms'], x['beds'], x['baths']), axis=1)
  
  # Fill baths zero values by adding beds and baths
  X.loc[(X['baths'] == 0), 'baths'] = 1
  
  # Fill lot_sf zero values by using lot_acres to calc
  def lotsf_calc(lotsf_val, lotacres_val):
    if lotsf_val == 0:
      total = lotacres_val * 43560
    else:
      total = lotsf_val
      
    return total
  
  X['lot_sf'] = X.apply(lambda x: lotsf_calc(x['lot_sf'], x['lot_acres']), axis=1)
  
  # Fill lot_acres zero values by using lot_sf to calc
  def lotacres_calc(lotacres_val, lotsf_val):
    if lotacres_val == 0:
      total = lotsf_val / 43560
    else:
      total = lotacres_val
      
    return total
  
  X['lot_acres'] = X.apply(lambda x: lotacres_calc(x['lot_acres'], x['lot_sf']), axis=1)
  
  # Engineer new feature mean_neighbor_price THIS TAKES 1.5 HOURS TO RUN
  # Output was saved to nhoods files and is loaded later to save time
  nhoods = X[['sf', 'longitude', 'latitude']]
  
  def neighbor_mean(sqft, source_latitude, source_longitude):
    
    source_latlong = source_latitude, source_longitude
    source_table = X[(X['sf'] >= (sqft * .9)) & (X['sf'] <= (sqft * 1.1))]
    target_table = pd.DataFrame(source_table, columns = ['latitude', 'longitude', 'year_sold', 'sale_price']) 

    def get_distance(row):
        target_latlong = row['latitude'], row['longitude']
        return get_geodesic_distance(target_latlong, source_latlong).meters

    target_table['distance'] = target_table.apply(get_distance, axis=1)

    # Get the nearest 3 locations
    nearest_target_table = target_table.sort_values(['year_sold', 'distance'], ascending=[False, True])[:3]

    return nearest_target_table['sale_price'].mean()/sqft

  #nhoods['mean_hood_ppsf'] = X.apply(lambda x: neighbor_mean(x['sf'], x['latitude'], x['longitude']), axis=1)
  #nhoods.to_csv('/content/nhoods.csv')
  
#   # Features with nan's that should be zero's
#   cols_with_nans = ['lot_sf', 'lot_acres', 'year_built', 'zoning']
#   for col in cols_with_nans:
#     X[col] = X[col].replace(np.nan, 0)
    
  # Engineer a price_sf column
  #X['price_sf'] = X['sale_price'] / X['sf']
  
  # Drop unneeded columns
  unneeded_columns = ['sale_date', 'on_market_date', 'city', 'state', 'street_no', 'street_name', 'street_suffix',
                      'day_on_market', 'month_on_market', 'year_on_market', 'month_sold', 'day_sold', 'orig_list_price', 'cdom',
                      'sf_source', 'area', 'sf_source_decs', 'lot_acres']
  X = X.drop(columns=unneeded_columns)
  
  return X

train = engineer_features(train)
test = engineer_features(test)

train.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-v

,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1260000,8.0,3.5,4,2691.0,NaN,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN",2013
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1075000,9.0,3.0,4,2437.0,3000.0,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0,2016
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1525000,9.0,5.0,5,2597.0,3000.0,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0,2017
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,715000,5.0,1.0,2,1312.0,NaN,1939,RH1,RGLR,"PVDW,PVSW",GARG,0,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK",2010
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1595000,8.0,4.0,4,3307.0,NaN,1951,NaN,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",0,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK",2012


In [0]:
rnhoods = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Jestimate/master/data/nhoods-train.csv')
tnhoods = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Jestimate/master/data/nhoods-test.csv')
train = pd.merge(train, rnhoods[['old_index', 'mean_hood_ppsf']], left_index=True, right_on='old_index')
test = pd.merge(test, tnhoods[['old_index', 'mean_hood_ppsf']], left_index=True, right_on='old_index')
train = train.drop(columns=['old_index'])
test = test.drop(columns=['old_index'])

In [217]:
print(train.shape)
train.head()

(17618, 28)


,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,mean_hood_ppsf
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1260000,8.0,3.5,4,2691.0,NaN,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN",2013,682.521987
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1075000,9.0,3.0,4,2437.0,3000.0,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0,2016,675.030776
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1525000,9.0,5.0,5,2597.0,3000.0,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0,2017,606.488256
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,715000,5.0,1.0,2,1312.0,NaN,1939,RH1,RGLR,"PVDW,PVSW",GARG,0,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK",2010,978.150406
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1595000,8.0,4.0,4,3307.0,NaN,1951,NaN,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",0,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK",2012,869.972785


In [218]:
print(test.shape)
test.head()

(1879, 28)


,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,mean_hood_ppsf
0,-122.509190,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121-2410",94121,1,SF District 1,1050,1 - Outer Richmond,3310000,8.0,4.0,4,3735.0,3959.0,1951,NaN,0,0,"ATCH,GARG,ATDR,INAC",ONST,3,3BLK,1BLK,2STR,"PNRM,CTYL,WATR,SFRN,OCEN,PARK,GRDN,TWNP",2018,887.996430
2,-122.509056,37.758010,23.01,"1516 Great Highway, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1830000,7.0,3.0,4,2120.0,3072.0,1948,RM1,"RGLR,OCNF",0,GARG,ONST,1,2BLK,2BLK,FULL,GRDN,2018,796.383648
4,-122.509020,37.758636,22.70,"1492 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1300000,3.0,1.0,2,1087.0,1245.0,1944,RM1,0,"PVDW,PVSW",ATCH,ONST,1,1BLK,1BLK,0,0,2018,1232.750690
5,-122.508980,37.773680,40.08,"784 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1300000,6.0,1.5,2,1326.0,2696.0,1942,RH2,"RGLR,LEVL,FNCD","PVDW,PVSW",GARG,ONST,2,1BLK,1BLK,"JR,2STR",0,2018,1045.751634
7,-122.508270,37.778465,225.66,"525 47th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1865000,6.0,2.0,3,1800.0,3000.0,1931,RH1,RGLR,"PVDW,PVSW","ATCH,GARG,ATDR,INAC",ONST,2,0,1BLK,"FULL,2STR",PRTL,2018,904.629630


In [219]:
pandas_profiling.ProfileReport(train)

Number of variables,29
Number of observations,17618
Total Missing (%),3.6%
Total size in memory,3.9 MiB
Average record size in memory,232.0 B
Numeric,15
Categorical,12
Boolean,0
Date,0
Text (Unique),0
Rejected,2


In [220]:
# Train
cutoff = 2017
temp=train.copy()
train = temp[temp['year_sold'] < 2017]
val  = temp[temp['year_sold'] >= 2017]
print(train.shape, val.shape, test.shape)

(15686, 28) (1932, 28) (1879, 28)


In [221]:
# Encode and fit a linear regression model

target = 'sale_price'

features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

pipeline = make_pipeline(
  ce.OrdinalEncoder(),
  SimpleImputer(strategy='mean'), 
  LinearRegression()
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)

# Print regression metrics for validation 
val_mse = mean_squared_error(y_val, y_pred)
val_rmse = np.sqrt(val_mse)
val_mae = mean_absolute_error(y_val, y_pred)
val_r2 = r2_score(y_val, y_pred)
print('Validation Mean Absolute Error:', val_mae)
print('Validation R^2:', val_r2)
print('\n')

ty_pred = pipeline.predict(X_test)

# Print regression metrics for test
test_mse = mean_squared_error(y_test, ty_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, ty_pred)
test_r2 = r2_score(y_test, ty_pred)
print('Test Mean Absolute Error:', test_mae)
print('Test R^2:', test_r2)
print('\n')

Validation Mean Absolute Error: 255212.31592534177
Validation R^2: 0.8027615072982429


Test Mean Absolute Error: 273451.47748818365
Test R^2: 0.7840406477031912




In [0]:
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}
# print(random_grid)
  
# pipeline = make_pipeline (
#   ce.OrdinalEncoder(), 
#   SimpleImputer(strategy='mean'),
#   RandomizedSearchCV(estimator = RandomForestRegressor(),
#                      param_distributions = random_grid,
#                      n_iter = 100,
#                      verbose=2,
#                      random_state=42,
#                      n_jobs = -1)
#   )

# pipeline.fit(X_train, y_train)

# pd.set_option('display.max_rows', 200)
# model = pipeline.named_steps['randomizedsearchcv']
# best = pd.Series(model.best_params_)
# print(best)

In [223]:
X_train.head()

,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,rooms,baths,beds,sf,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,mean_hood_ppsf
0,-122.509650,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,8.0,3.5,4,2691.0,NaN,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN",2013,682.521987
1,-122.509290,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,9.0,3.0,4,2437.0,3000.0,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0,2016,675.030776
3,-122.509240,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,5.0,1.0,2,1312.0,NaN,1939,RH1,RGLR,"PVDW,PVSW",GARG,0,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK",2010,978.150406
4,-122.509190,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,8.0,4.0,4,3307.0,NaN,1951,NaN,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",0,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK",2012,869.972785
5,-122.509186,37.761005,23.69,"1362 La Playa St, San Francisco, CA 94122-1019",94122,2,SF District 2,2030,2 - Outer Sunset,4.0,1.0,3,1168.0,NaN,1947,NaN,"RGLR,LEVL",PVDW,"ATCH,GARG,ATDR",0,2,0,1BLK,ATAC,0,2009,866.438356


In [224]:
final = test.copy()
final['prediction'] = ty_pred
final['difference'] = final['sale_price'] - final['prediction']
final['prediction'] = final['prediction'].astype(int)
final['difference'] = final['difference'].astype(int)
final.head()

,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,mean_hood_ppsf,prediction,difference
0,-122.509190,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121-2410",94121,1,SF District 1,1050,1 - Outer Richmond,3310000,8.0,4.0,4,3735.0,3959.0,1951,NaN,0,0,"ATCH,GARG,ATDR,INAC",ONST,3,3BLK,1BLK,2STR,"PNRM,CTYL,WATR,SFRN,OCEN,PARK,GRDN,TWNP",2018,887.996430,3321248,-11248
2,-122.509056,37.758010,23.01,"1516 Great Highway, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1830000,7.0,3.0,4,2120.0,3072.0,1948,RM1,"RGLR,OCNF",0,GARG,ONST,1,2BLK,2BLK,FULL,GRDN,2018,796.383648,1845460,-15460
4,-122.509020,37.758636,22.70,"1492 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1300000,3.0,1.0,2,1087.0,1245.0,1944,RM1,0,"PVDW,PVSW",ATCH,ONST,1,1BLK,1BLK,0,0,2018,1232.750690,1388961,-88961
5,-122.508980,37.773680,40.08,"784 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1300000,6.0,1.5,2,1326.0,2696.0,1942,RH2,"RGLR,LEVL,FNCD","PVDW,PVSW",GARG,ONST,2,1BLK,1BLK,"JR,2STR",0,2018,1045.751634,1535014,-235014
7,-122.508270,37.778465,225.66,"525 47th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1865000,6.0,2.0,3,1800.0,3000.0,1931,RH1,RGLR,"PVDW,PVSW","ATCH,GARG,ATDR,INAC",ONST,2,0,1BLK,"FULL,2STR",PRTL,2018,904.629630,1806484,58515


In [225]:
final.shape

(1879, 30)

In [0]:
# Calculate the agent_pred using the mean neighborhood price/sf * square footage of the house
def agent_calc(nhood, sqft):
  a_pred = nhood_dict[nhood] * sqft
  return a_pred

final['pred_percent'] = final['difference']/final['prediction']

In [227]:
final.head()

,longitude,latitude,elevation,full_address,zip,district_no,district_desc,nid,neighborhood,sale_price,rooms,baths,beds,sf,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,mean_hood_ppsf,prediction,difference,pred_percent
0,-122.509190,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121-2410",94121,1,SF District 1,1050,1 - Outer Richmond,3310000,8.0,4.0,4,3735.0,3959.0,1951,NaN,0,0,"ATCH,GARG,ATDR,INAC",ONST,3,3BLK,1BLK,2STR,"PNRM,CTYL,WATR,SFRN,OCEN,PARK,GRDN,TWNP",2018,887.996430,3321248,-11248,-0.003387
2,-122.509056,37.758010,23.01,"1516 Great Highway, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1830000,7.0,3.0,4,2120.0,3072.0,1948,RM1,"RGLR,OCNF",0,GARG,ONST,1,2BLK,2BLK,FULL,GRDN,2018,796.383648,1845460,-15460,-0.008377
4,-122.509020,37.758636,22.70,"1492 La Playa St, San Francisco, CA 94122",94122,2,SF District 2,2030,2 - Outer Sunset,1300000,3.0,1.0,2,1087.0,1245.0,1944,RM1,0,"PVDW,PVSW",ATCH,ONST,1,1BLK,1BLK,0,0,2018,1232.750690,1388961,-88961,-0.064049
5,-122.508980,37.773680,40.08,"784 48th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1300000,6.0,1.5,2,1326.0,2696.0,1942,RH2,"RGLR,LEVL,FNCD","PVDW,PVSW",GARG,ONST,2,1BLK,1BLK,"JR,2STR",0,2018,1045.751634,1535014,-235014,-0.153102
7,-122.508270,37.778465,225.66,"525 47th Ave, San Francisco, CA 94121",94121,1,SF District 1,1050,1 - Outer Richmond,1865000,6.0,2.0,3,1800.0,3000.0,1931,RH1,RGLR,"PVDW,PVSW","ATCH,GARG,ATDR,INAC",ONST,2,0,1BLK,"FULL,2STR",PRTL,2018,904.629630,1806484,58515,0.032392


In [228]:
pred_five_percent = (final['pred_percent'][(final['pred_percent'] >= -.05) & (final['pred_percent'] <= .05)].count())/final.shape[0]
print('Prediction Error Within 5% -', pred_five_percent)

Prediction Error Within 5% - 0.2368281000532198


In [0]:
# mae = np.sum(np.absolute((final['agent_diff'])))/final['agent_diff'].count()
# print(mae)

In [0]:
# data = final[['full_address', 'neighborhood', 'nid', 'sale_price', 'prediction', 'difference']]
# data = data.drop_duplicates()

# data.head()

In [0]:
# data.shape

In [0]:
# data.to_csv('/content/display_data.csv')